In [23]:
import os
import xmltodict
import csv

# CSVファイルの出力先
output_file = "output_data.csv"

# 抽出したい文字列
user_name_to_extract = "HANBAI"
timestamp_to_extract_s = "2023-12-13T01:00:25.744452Z"
timestamp_to_extract_e = "2023-12-13T01:09:50.744452Z"

# 抽出する要素の指定
elements_to_extract = ['Session_Id','EntryId','DB_User', 'Userhost','Extended_Timestamp','Comment_Text','DBID']
#elements_to_extract = ['Session_Id','EntryId','DB_User', 'Userhost','Extended_Timestamp','Comment_Text','DBID','Sql_Text']

# CSVファイルへの追記関数
def append_to_csv(extracted_data, output_file, processed_rows):
    with open(output_file, 'a', newline='') as csvfile:
        fieldnames = elements_to_extract
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # ヘッダーの追記
        if csvfile.tell() == 0:  
            writer.writeheader()

        # CSVファイルへの書き込み
        row = {}
        for field in fieldnames:
            row[field] = extracted_data.get(field, '')  
        writer.writerow(row)

# 処理した行数をカウントする変数
processed_rows = 0

# 指定されたフォルダ内のすべてのXMLファイルを処理
folder_path = "./data"
for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):
        # XMLファイルを読み込み、XML_DATAに追加
        with open(os.path.join(folder_path, filename), "r") as xml_file:
            XML_DATA = xml_file.read()

        parsed = xmltodict.parse(XML_DATA)

        print('#############',filename)
        # 指定された文字列の場合にデータを抽出する
        for rdata in parsed['Audit']['AuditRecord']:
            extracted_data = {}
            for element in elements_to_extract:
                extracted_data[element] = rdata.get(element, '')  

            if (rdata['DB_User'] == user_name_to_extract) and \
                rdata['Extended_Timestamp'] >= timestamp_to_extract_s and \
                rdata['Extended_Timestamp'] <= timestamp_to_extract_e:
                # CSVファイルへの追記
                append_to_csv(extracted_data, output_file, processed_rows)
                processed_rows += 1  

                #print(f"{', '.join(elements_to_extract)} data for {user_name_to_extract} from {filename} has been appended to {output_file}")

# 処理した行数を出力
print(f"Total {processed_rows} rows have been appended to {output_file}.")
# 処理した行数もCSVファイルに追記
with open(output_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f"Processed Rows:{processed_rows} finished!"])


############# DWH_ora_23288_20231213100338343742161532.xml
############# DWH_ora_24378_20231213100830368940366409.xml
############# DWH_ora_24203_20231213100736371452344179.xml
Total 330 rows have been appended to output_data.csv.


In [8]:
import pandas as pd
# CSVファイルの読み込み
df = pd.read_csv("output_data.csv")
df.head

<bound method NDFrame.head of                        Session_Id  EntryId DB_User Userhost  \
0                       246681550      1.0  HANBAI    eigyt   
1                       246681550      2.0  HANBAI    eigyt   
2                       246681550      3.0  HANBAI    eigyt   
3                       246681550      4.0  HANBAI    eigyt   
4                       246681550      5.0  HANBAI    eigyt   
..                            ...      ...     ...      ...   
988                     246681746     12.0  HANBAI    eigyt   
989                     246681746     13.0  HANBAI    eigyt   
990                     246681746     14.0  HANBAI    eigyt   
991                     246681746     15.0  HANBAI    eigyt   
992  Processed Rows:330 finished!      NaN     NaN      NaN   

              Extended_Timestamp  \
0    2023-12-13T01:03:38.343978Z   
1    2023-12-13T01:03:38.460576Z   
2    2023-12-13T01:03:38.460729Z   
3    2023-12-13T01:03:38.460827Z   
4    2023-12-13T01:03:38.460930Z 

In [7]:
import plotly.express as px

# Plotlyを使用してデータをプロット
fig = px.scatter(df, x='Extended_Timestamp', y='EntryId', title=f"count(EntryId)",color='Session_Id')
fig.show()


In [18]:
#xmlparseしたリスト parsed
print(type(parsed['Audit']['AuditRecord']),len(parsed['Audit']['AuditRecord']))
parsed['Audit']['AuditRecord'][0]

<class 'list'> 16


{'Audit_Type': '1',
 'Session_Id': '246681746',
 'StatementId': '1',
 'EntryId': '1',
 'Extended_Timestamp': '2023-12-13T01:07:36.371653Z',
 'DB_User': 'HANBAI',
 'OS_User': 'oracle',
 'Userhost': 'eigyt',
 'OS_Process': '24203',
 'Instance_Number': '0',
 'Action': '100',
 'TransactionId': '0000000000000000',
 'Returncode': '0',
 'Comment_Text': 'Authenticated by: DATABASE; Client address: (ADDRESS=(PROTOCOL=tcp)(HOST=10.241.200.225)(PORT=53256)); DBLINK_INFO: (SOURCE_GLOBAL_NAME=orcl.118691313)',
 'Priv_Used': '5',
 'DBID': '2082544068'}

In [27]:
sql2 = parsed['Audit']['AuditRecord'][3]['Sql_Text']

In [30]:
!pip install sqlparse

  Obtaining dependency information for sqlparse from https://files.pythonhosted.org/packages/98/5a/66d7c9305baa9f11857f247d4ba761402cea75db6058ff850ed7128957b7/sqlparse-0.4.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.0 MB/s eta 0:00:00


In [31]:
import sqlparse

sql_1 = "select * from t1"
parsed_queries = sqlparse.parse(sql_1)
parsed_query = parsed_queries[0]
list(parsed_query.flatten())

[<DML 'select' at 0x7F0C05BC9A20>,
 <Whitespace ' ' at 0x7F0C05BC9A80>,
 <Wildcard '*' at 0x7F0C05BC9AE0>,
 <Whitespace ' ' at 0x7F0C05BC9B40>,
 <Keyword 'from' at 0x7F0C05BC9BA0>,
 <Whitespace ' ' at 0x7F0C05BC9C00>,
 <Name 't1' at 0x7F0C05BC9C60>]